<a href="https://colab.research.google.com/github/beyzaatasever/als-movie-recommender/blob/main/collabrative_filtering_ipynb_adl%C4%B1_dosyan%C4%B1n_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')


print(movies_df.head())



In [ ]:
import pandas as pd


films_df = pd.read_csv('films.csv')


print(films_df.head())
films_df.head()


In [ ]:

print("Movies DataFrame:")
print(movies_df.head())

print("\nRatings DataFrame:")
print(ratings_df.head())

print("\nFilms DataFrame:")
print(films_df.head())

In [ ]:
# extract year column
import pandas as pd
import re  # 're' modülünü içe aktarıyoruz
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    return None

movies_df['year'] = movies_df['title'].apply(extract_year)

# check year column
print("Movies DataFrame with extracted year:")
print(movies_df.head())

movies_df = movies_df.dropna(subset=['year']) #na leri kaldırdım
#delete NaN values
movies_df['year'] = movies_df['year'].astype(int)

# check new year column
print("Movies DataFrame with extracted year:")
print(movies_df.head())

In [ ]:

print("Movies DataFrame:")
print(movies_df.head())

print("\nFilms DataFrame:")
print(films_df.head())

In [ ]:

films_df['Name'] = films_df['Name'].str.lower().str.strip()
movies_df['title'] = movies_df['title'].str.lower().str.strip()

print("Cleaned Movies DataFrame with integer year:")
print(movies_df.head())
print("\nFilms DataFrame:")
print(films_df.head())


In [ ]:
# merged films
merged_df = pd.merge(films_df, movies_df, left_on=['Name', 'Year'], right_on=['title', 'year'], how='inner')

In [ ]:
# check frames
print("Films DataFrame:")
print(films_df[['Name', 'Year']].head())
print(films_df.dtypes)

print("\nMovies DataFrame:")
print(movies_df[['title', 'year']].head())
print(movies_df.dtypes)


In [ ]:

films_selected = films_df[['Name', 'Year']]

print("Selected Films DataFrame:")
print(films_selected)

films_list = films_selected.values.tolist()


print("Films List:")
print(films_list)


In [ ]:
# found matched films
def find_matching_movie(name, year, movies_df):
    matching_movies = movies_df[(movies_df['year'] == year) & (movies_df['title'].str.contains(name, case=False, na=False))]
    return matching_movies

# found for al films matched
matched_movies = []
for name, year in films_list:
    matched = find_matching_movie(name, year, movies_df)
    if not matched.empty:
        matched_movies.append(matched)

# Turned matched films into dataframes
if matched_movies:
    matched_movies_df = pd.concat(matched_movies)
else:
    matched_movies_df = pd.DataFrame()

print("\nMatched Movies DataFrame:")
print(matched_movies_df)


In [ ]:
# lets give new ID to the user
user_id = 9999

# adding ratings for user
user_ratings = matched_movies_df[['movieId']].copy()
user_ratings['userId'] = user_id
user_ratings['rating'] = 5  # added 5 stars

# adding new rating to old ones
all_ratings = pd.concat([ratings_df[['userId', 'movieId', 'rating']], user_ratings])

print("\nAll Ratings DataFrame (Yeni Kullanıcı Ratingleriyle):")
print(all_ratings.tail())


In [ ]:


from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName("movie-recommendation").getOrCreate()

all_ratings_spark = spark.createDataFrame(all_ratings)


(training, test) = all_ratings_spark.randomSplit([0.8, 0.2])


als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop",
    rank=10,
    maxIter=15,
    regParam=0.1
)


model = als.fit(training)


predictions = model.transform(test)

# Tahminleri ve gerçek değerleri göster
predictions.show()

# RMSE'yi hesaplayalım
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


# Calculating MAE
evaluator_mae = RegressionEvaluator(
    metricName="mae",
    labelCol="rating",
    predictionCol="prediction"
)

mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error = " + str(mae))


# Make recomandations
user_subset = all_ratings_spark.filter(all_ratings_spark.userId == user_id)
recommendations = model.recommendForUserSubset(user_subset, numItems=10)

recommendations_df = recommendations.toPandas()
recommended_movie_ids = [row.movieId for row in recommendations_df.iloc[0]['recommendations']]


recommended_movies = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]
print("\nRecommended Movies:")
print(recommended_movies[['movieId', 'title']])
